In [1]:
import cv2
import os
import threading
import time
from tkinter import *
from PIL import Image, ImageTk

class CameraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Camera Capture App")
        self.root.geometry("600x600")  # Adjusted window size to make buttons more visible
        self.root.configure(bg="#333333")

        self.folder_name = StringVar(value="Right")
        self.file_prefix = StringVar(value="direction")
        self.interval = 1
        self.width = 1920
        self.height = 1080
        self.num_photos = IntVar(value=10)  # Default number of photos

        self.header = Label(root, text="Camera Capture Application", bg="#444444", fg="#ffffff", font=("Helvetica", 14, "bold"))
        self.header.pack(pady=5)

        self.settings_frame = Frame(root, bg="#333333")
        self.settings_frame.pack(pady=10)

        self.folder_label = Label(self.settings_frame, text="Folder Name:", bg="#333333", fg="#ffffff", font=("Helvetica", 10))
        self.folder_label.grid(row=0, column=0, padx=5, pady=5)
        self.folder_entry = Entry(self.settings_frame, textvariable=self.folder_name, font=("Helvetica", 10))
        self.folder_entry.grid(row=0, column=1, padx=5, pady=5)

        self.prefix_label = Label(self.settings_frame, text="File Prefix:", bg="#333333", fg="#ffffff", font=("Helvetica", 10))
        self.prefix_label.grid(row=1, column=0, padx=5, pady=5)
        self.prefix_entry = Entry(self.settings_frame, textvariable=self.file_prefix, font=("Helvetica", 10))
        self.prefix_entry.grid(row=1, column=1, padx=5, pady=5)

        self.num_photos_label = Label(self.settings_frame, text="Number of Photos:", bg="#333333", fg="#ffffff", font=("Helvetica", 10))
        self.num_photos_label.grid(row=2, column=0, padx=5, pady=5)
        self.num_photos_entry = Entry(self.settings_frame, textvariable=self.num_photos, font=("Helvetica", 10))
        self.num_photos_entry.grid(row=2, column=1, padx=5, pady=5)

        self.camera_frame = Label(root, bg="#333333", width=640, height=360)  # Reduced size
        self.camera_frame.pack(pady=10)

        self.button_frame = Frame(root, bg="#444444")
        self.button_frame.pack(pady=10)

        self.start_button = Button(self.button_frame, text="Start", command=self.start_capture, bg="#28a745", fg="#ffffff", font=("Helvetica", 12, "bold"), width=10)
        self.start_button.pack(side=LEFT, padx=10)

        self.end_button = Button(self.button_frame, text="End", command=self.end_capture, bg="#dc3545", fg="#ffffff", font=("Helvetica", 12, "bold"), width=10)
        self.end_button.pack(side=LEFT, padx=10)  # Adjusted alignment

        self.capture = False
        self.cap = cv2.VideoCapture(1)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3840)  # Set to high resolution
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)

        # Delay the first call to update_camera_frame
        self.root.after(100, self.update_camera_frame)

    def update_camera_frame(self):
        if self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(cv2image)
                imgtk = ImageTk.PhotoImage(image=img)
                self.camera_frame.imgtk = imgtk
                self.camera_frame.configure(image=imgtk)

        self.root.after(10, self.update_camera_frame)

    def start_capture(self):
        self.capture = True
        self.capture_thread = threading.Thread(target=self.capture_images)
        self.capture_thread.start()

    def capture_images(self):
        folder_path = self.folder_name.get()
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        existing_files = os.listdir(folder_path)
        existing_indexes = [int(f.split('_')[-1].split('.')[0]) for f in existing_files if f.startswith(self.file_prefix.get()) and f.endswith('.png')]
        if existing_indexes:
            count = max(existing_indexes) + 1
        else:
            count = 0

        num_photos_to_take = self.num_photos.get()

        for _ in range(num_photos_to_take):
            if not self.capture:
                break
            ret, frame = self.cap.read()
            if ret:
                # Calculate scaling factors
                h, w = frame.shape[:2]
                scale_w = self.width / w
                scale_h = self.height / h
                scale = max(scale_w, scale_h)  # Use the larger scale to avoid padding

                # Resize the frame using high-quality interpolation
                new_w = int(w * scale)
                new_h = int(h * scale)
                resized_frame = cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)

                # Center crop to 1920x1080
                start_x = (new_w - self.width) // 2
                start_y = (new_h - self.height) // 2
                cropped_frame = resized_frame[start_y:start_y + self.height, start_x:start_x + self.width]

                img_name = os.path.join(folder_path, f"{self.file_prefix.get()}_{count:04d}.png")
                cv2.imwrite(img_name, cropped_frame)
                count += 1
                time.sleep(self.interval)

        self.capture = False  # Automatically stop after capturing the required number of photos

    def end_capture(self):
        self.capture = False
        if self.capture_thread.is_alive():
            self.capture_thread.join()
        self.cap.release()
        self.root.quit()

if __name__ == "__main__":
    root = Tk()
    app = CameraApp(root)
    root.mainloop()
